In [ ]:
import pandas as pd
import datetime
import numpy as np

# Linkedin Data from Kaggle

## Data Preprocessing
### Converting Unix Timestamps
#### Overview
The convert_unix_to_ddmmyyyy function transforms Unix timestamps (in milliseconds) into a human-readable date format (dd-mm-yyyy). This step ensures the timestamps are interpretable and consistent for analysis and visualization.

### Function Parameters
unix_time (pd.Series or list): A collection of Unix timestamps in milliseconds to be converted.
### Key Steps
Adjusting for Datetime Conversion: Converts Unix timestamps from milliseconds to seconds for compatibility with datetime libraries.
Datetime Transformation: Uses pandas to handle conversion to datetime objects while managing invalid timestamps.
Formatting: Outputs the dates in the dd-mm-yyyy format for readability.

In [22]:
data = pd.read_csv("job postings 2023 24/postings.csv")

In [ ]:
data.columns

In [ ]:
data.original_listed_time[:2]

In [ ]:
def convert_unix_to_ddmmyyyy(unix_time):
    # Convert from milliseconds to seconds
    unix_time_seconds = unix_time / 1000.0
    
    # Convert to datetime and handle errors
    formatted_dates = pd.to_datetime(unix_time_seconds, unit='s', errors='coerce')
    
    # Format as 'dd-mm-yyyy'
    return formatted_dates.dt.strftime('%d-%m-%Y')

In [ ]:
raw_data = data.copy()

In [ ]:
raw_data['original_listed_time_mod'] = convert_unix_to_ddmmyyyy(raw_data['original_listed_time'])

In [ ]:
raw_data['original_listed_time_mod'] =pd.to_datetime(raw_data['original_listed_time_mod'], format='%d-%m-%Y', errors='coerce')

In [ ]:
raw_data['original_listed_time_mod'].min()

In [ ]:
raw_data['original_listed_time_mod'].max()

# Fetching LinkedIn Jobs using ScrapingDog API

## Overview
This script fetches job postings from LinkedIn using the ScrapingDog API. It retrieves data for specific fields (job categories), locations (geo IDs), and pages, combining the results into a unified dataset.


## Function Definition: `fetch_all_linkedin_jobs`

### Purpose
The `fetch_all_linkedin_jobs` function retrieves LinkedIn job data for specified parameters (fields, geo IDs, and pages) using API calls and compiles it into a single dataset.

### Parameters
- **`api_key` (str)**: Your API key for authentication with ScrapingDog.
- **`fields` (list of str)**: Job categories or search terms to filter jobs (e.g., "data science", "machine learning").
- **`geoids` (list of str)**: Geographic region codes (e.g., NV, Colorado, California).
- **`pages` (list of str)**: Page numbers to fetch paginated results.

### Return Value
- **List of JSON objects**: Combined data retrieved from all API calls.


## Code

```python
import requests
import pandas as pd  # Required to save data as CSV

def fetch_all_linkedin_jobs(api_key, fields, geoids, pages):
    """
    Fetches LinkedIn job postings using the ScrapingDog API.

    Parameters:
    - api_key (str): API key for authentication.
    - fields (list): List of job categories to fetch (e.g., ["java", "data science"]).
    - geoids (list): List of geographic region IDs to fetch jobs from.
    - pages (list): List of pages to iterate over for paginated results.

    Returns:
    - list: A combined list of JSON objects with job data.
    """
    url = "https://api.scrapingdog.com/linkedinjobs/"
    all_data = []  # Store data from all requests

    # Iterate over each combination of field, geoid, and page
    for field in fields:
        for geoid in geoids:
            for page in pages:
                params = {
                    "api_key": api_key,
                    "field": field,
                    "geoid": geoid,
                    "page": page
                }
                
                response = requests.get(url, params=params)
                
                if response.status_code == 200:
                    # Append data from each request to the list
                    all_data.extend(response.json())
                else:
                    print(f"Request failed for field: {field}, geoid: {geoid}, page: {page} with status code: {response.status_code}")

    return all_data

# Data Preprocessing

## Filtering Data

**Filtering Function Overview**
The filter_data function dynamically filters a DataFrame based on a specified list of columns and corresponding conditions. This flexibility allows for customized data extraction based on various criteria, such as dates, maximum or minimum values, and frequent items.

**Function Parameters**
df (pd.DataFrame): The input DataFrame containing the data you want to filter.

columns (list): A list of column names to apply filters on. Each column corresponds to a condition in the conditions list.

conditions (list): A list of conditions, where each element is a tuple specifying the operation and the value for filtering. Each tuple matches one of the columns in the columns list.


In [23]:
import pandas as pd

def filter_data(df, columns, conditions):
    """
    Filters a DataFrame based on a list of columns and their conditions.

    Parameters:
    - df (pd.DataFrame): The input DataFrame to filter.
    - columns (list): List of column names to apply filters on.
    - conditions (list): List of conditions where each element is a tuple in the format:
                         (condition, value), for example ('>', '2024-04-20').
                         Each tuple should correspond to a column in the columns list.
    
    Returns:
    - pd.DataFrame: A filtered DataFrame.
    """
    # Loop over each column and its corresponding condition
    for column, (condition, value) in zip(columns, conditions):
        if condition == '>':
            df = df[df[column] > value]
        elif condition == '<':
            df = df[df[column] < value]
        elif condition == '==':
            df = df[df[column] == value]
        elif condition == '>=':
            df = df[df[column] >= value]
        elif condition == '<=':
            df = df[df[column] <= value]
        elif condition == '!=':
            df = df[df[column] != value]
        elif condition == 'max':
            max_value = df[column].max()
            df = df[df[column] == max_value]
        elif condition == 'min':
            min_value = df[column].min()
            df = df[df[column] == min_value]
        elif condition == 'top_n':
            top_n_counts = df[column].value_counts().nlargest(value)
            df = df[df[column].isin(top_n_counts.index)]
        elif condition == 'top_n':
            top_n_counts = df[column].value_counts()#.nlargest(value)
            df = df[df[column].isin(top_n_counts.index)]
        else:
            print(f"Invalid condition '{condition}' for column '{column}'")
    
    return df


## Filtering Scraped Data by Date

### Overview
This process filters job posting data based on specific date criteria, ensuring only relevant records are retained for further analysis.

### Steps
1. Load Data: Load the scraped job data into a pandas DataFrame.
2. Prepare Date Column: Convert the job_posting_date column to datetime format, handling any invalid entries gracefully.
3. Define Filters: Specify the column (job_posting_date) and condition (> a specific date) to filter postings after a chosen date.
4. Apply Filter: Use the filter_data function to extract rows meeting the criteria.
5. Save and Review: Save the filtered dataset to a CSV file and display the results for verification.

This ensures the dataset contains only job postings after the specified date, optimizing it for further use.

In [ ]:
# Load the data
scraped_data = pd.read_csv("C:/Users/DELL/Downloads/261024_Job_Data.csv")

# Convert date column to datetime if filtering by date
scraped_data['job_posting_date'] = pd.to_datetime(scraped_data['job_posting_date'], errors='coerce')

# Specify columns and conditions
columns = ['job_posting_date']
conditions = [('>', '2024-04-20')]

# Apply the filter function
filtered_scraped_data = filter_data(scraped_data, columns, conditions)

# Save the filtered data
filtered_scraped_data.to_csv("filtered_scraped_job_data.csv", index=False)

# Print the filtered data
print(filtered_scraped_data)

## Filtering Current Kaggle Data to check top N job positions

This code processes a dataset of job postings to identify and extract the most common job titles. Here's a breakdown of its functionality:

1. **Load Dataset**:
The dataset postings.csv is loaded into a pandas DataFrame called kaggledata.

2. **Top-N Filtering Function**:
The filter_data function (previously defined) is used to filter the top 50 most frequent job titles (title column) based on their frequency in the dataset. The filtering uses the top_n condition to identify and retain rows corresponding to the most frequently occurring job titles.

3. **Get Top Job Roles**:
The filtered dataset is stored in the variable top_15_job_roles, which now contains rows corresponding to the top job titles.

4. **Unique Job Titles**:
The .unique() method is applied to the title column of top_15_job_roles to extract the unique job titles among the top 50, showing a list of distinct job roles.

**Purpose**:
The code identifies and highlights the most common job titles in the dataset, which can be useful for trend analysis, visualization, or reporting the popularity of specific roles.

In [ ]:
import pandas as pd
kaggledata = pd.read_csv("job postings 2023 24/postings.csv")


In [ ]:
def filter_top_n_values(df, column, n):
    # Get the top n most frequent values in the specified column
    top_n_values = df[column].value_counts()#.nlargest(n)
    return top_n_values
columns = ['title']
conditions = [('top_n',50)]
top_15_job_roles = filter_data(kaggledata, columns, conditions)

print(top_15_job_roles)

In [ ]:
top_15_job_roles['title'].unique()

## Creating a Data Cube for Job Analysis
This code generates a data cube from a dataset of job postings, summarizing key metrics across multiple dimensions. Here's what it does:

**Pivot Table Creation**:
The pivot_table method is used to aggregate the dataset (kaggledata) based on specified values and dimensions.
Metrics such as salaries, views, and applications are calculated using aggregation functions, grouped by dimensions like location, company name, job title, work type, remote flexibility, and experience level.

**Metrics**:
med_salary: Average median salary (mean).
max_salary: Highest salary value (max).
min_salary: Lowest salary value (min).
views: Total job views (sum).
applies: Total number of applications (sum).
normalized_salary: Average normalized salary (mean).

**Dimensions**:
location: Geographic location of the job.
company_name: Name of the hiring company.
title: Job title or role.
formatted_work_type: Type of employment (e.g., full-time, part-time).
remote_allowed: Whether the job allows remote work.
formatted_experience_level: Required experience level.

**Reset Index**:
The reset_index method is used to flatten the hierarchical index created by the pivot table, making it a regular DataFrame for easier manipulation and analysis.

**Output the Data Cube**:
The resulting data_cube is printed to display aggregated insights about the dataset.
An optional CSV file (job_data_cube.csv) is saved for external analysis or reporting.

**Purpose**:
This code aggregates and organizes job posting data, enabling multi-dimensional analysis of salaries, views, and applications across factors like location, role, and experience level. The resulting data cube simplifies trend identification, comparisons, and decision-making.

In [ ]:
data_cube = kaggledata.pivot_table(
    values=[
        'med_salary', 'max_salary', 'min_salary', 'views', 'applies', 'normalized_salary'
    ],
    index=[
        'location', 'company_name', 'title', 'formatted_work_type', 'remote_allowed', 'formatted_experience_level'
    ],
    aggfunc={
        'med_salary': 'mean',           # Average median salary
        'max_salary': 'max',            # Maximum salary
        'min_salary': 'min',            # Minimum salary
        'views': 'sum',                 # Total views
        'applies': 'sum',               # Total applications
        'normalized_salary': 'mean'     # Average normalized salary
    }
).reset_index()

# Display the resulting data cube
print("Data Cube:")
print(data_cube)

# Optional: Save to CSV for further analysis
data_cube.to_csv('job_data_cube.csv', index=False)

# Dynamic Data Cube with Filters

## Overview

The `dynamic_data_cube` function is designed to extract and aggregate metrics from a DataFrame based on user-specified filters such as company name, job title, location, and work type.

## Key Functionality

### Filtering Data
The function applies filters to the input DataFrame (`df`) based on the following optional parameters:
- **`company_name`**: Filters rows with a specific company name.
- **`title`**: Filters rows for a particular job title.
- **`location`**: Filters rows by location.
- **`work_type`**: Filters rows by work type (e.g., remote, on-site).

### Validation
If the filtered DataFrame is empty (i.e., no data matches the provided filters), the function returns a message indicating no results were found.

### Aggregation
For the filtered data, the function calculates the following metrics:
- **`Total Job Listings`**: Count of job listings in the filtered dataset.
- **`Max Salary`**: The highest salary from the filtered data.
- **`Min Salary`**: The lowest salary from the filtered data.
- **`Average Median Salary`**: The average of the median salaries in the filtered data.
- **`Total Views`**: The total number of views across all filtered job listings.
- **`Total Applications`**: The total number of applications received for the filtered job listings.
- **`Average Normalized Salary`**: The average normalized salary in the filtered data.


In [ ]:
def dynamic_data_cube(df, company_name=None, title=None, location=None, work_type=None):
    filtered_df = df.copy()
    
    # Apply filters based on user input
    if company_name:
        filtered_df = filtered_df[filtered_df['company_name'] == company_name]
    if title:
        filtered_df = filtered_df[filtered_df['title'] == title]
    if location:
        filtered_df = filtered_df[filtered_df['location'] == location]
    if work_type:
        filtered_df = filtered_df[filtered_df['formatted_work_type'] == work_type]
    
    # Check if filtered DataFrame is not empty
    if filtered_df.empty:
        return "No data found for the specified filters."
    
    # Aggregate metrics for the filtered data
    result = {
        'Total Job Listings': len(filtered_df),
        'Max Salary': filtered_df['max_salary'].max(),
        'Min Salary': filtered_df['min_salary'].min(),
        'Average Median Salary': filtered_df['med_salary'].mean(),
        'Total Views': filtered_df['views'].sum(),
        'Total Applications': filtered_df['applies'].sum(),
        'Average Normalized Salary': filtered_df['normalized_salary'].mean()
    }
    
    return result


In [ ]:
print(dynamic_data_cube(kaggledata, company_name="ServiceNow"))

In [ ]:
print(dynamic_data_cube(kaggledata, title="Full Stack Java Developer"))

# Merging Job Data with Industry Information

## Overview

This code demonstrates how to merge two datasets: one containing job postings and another containing industry information for companies. The datasets are merged based on the `company_id` column to provide a combined dataset with job information alongside the corresponding industry details.

## Steps

1. **Load the Datasets**
   - The `jobs_df` dataset contains job postings.
   - The `industries_df` dataset contains information about company industries.

   Both datasets are loaded from CSV files using `pd.read_csv()`.

2. **Merge the Datasets**
   - The two datasets are merged on the `company_id` column using a **left join**. This ensures that all job postings are retained, and industry information is added wherever available.

3. **Save the Merged Dataset**
   - After merging, the combined dataset is saved to a new CSV file called `merged_jobs_with_industries.csv`.

4. **Confirmation Message**
   - A message is printed to indicate the successful completion of the merge and the addition of the industry column.

## Code

```python
import pandas as pd

# Load the datasets
jobs_df = pd.read_csv("C:/Users/DELL/Linkedin-Job-Market-Analysis-using-ML/LinkedIn Scraper/job postings 2023 24/postings.csv") 
industries_df = pd.read_csv('C:/Users/DELL/Linkedin-Job-Market-Analysis-using-ML/LinkedIn Scraper/job postings 2023 24/companies/company_industries.csv') 

# Merge the datasets on the company_id column
merged_df = jobs_df.merge(industries_df, on='company_id', how='left')  # Perform a left join

# Save the merged dataset to a new CSV file
merged_df.to_csv('merged_jobs_with_industries.csv', index=False)

print("Merge completed. The industries column has been added.")


# Dynamic Data Cube with Industries Included

## Overview

This function generates a dynamic data cube by filtering job postings based on various criteria, including the company name, job title, location, work type, and industry. After applying the filters, it aggregates key metrics such as salary, views, and applications.

## Functionality

1. **Filter the Data**  
   The function allows users to filter the data based on multiple criteria:
   - **Company Name**: Filters by the company offering the job.
   - **Title**: Filters by job title.
   - **Location**: Filters by job location.
   - **Work Type**: Filters by work type (e.g., full-time, part-time).
   - **Industry**: Filters by the job's industry, with a case-insensitive search.

2. **Aggregation of Key Metrics**  
   After applying the filters, the following metrics are calculated:
   - Total job listings
   - Maximum salary
   - Minimum salary
   - Average median salary
   - Total views
   - Total applications
   - Average normalized salary

3. **Return Results**  
   If no data matches the filter criteria, a message is returned stating that no data was found. Otherwise, the function returns the aggregated results.

In [7]:
def dynamic_data_cube(df, company_name=None, title=None, location=None, work_type=None, industry=None):
    filtered_df = df.copy()
    
    # Apply filters based on user input
    if company_name:
        filtered_df = filtered_df[filtered_df['company_name'] == company_name]
    if title:
        filtered_df = filtered_df[filtered_df['title'] == title]
    if location:
        filtered_df = filtered_df[filtered_df['location'] == location]
    if work_type:
        filtered_df = filtered_df[filtered_df['formatted_work_type'] == work_type]
    if industry:
        filtered_df = filtered_df[filtered_df['industry'].str.contains(industry, case=False, na=False)]  # Case-insensitive
    # Check if filtered DataFrame is not empty
    if filtered_df.empty:
        return "No data found for the specified filters."
    
    # Aggregate metrics for the filtered data
    result = {
        'Total Job Listings': len(filtered_df),
        'Max Salary': filtered_df['max_salary'].max(),
        'Min Salary': filtered_df['min_salary'].min(),
        'Average Median Salary': filtered_df['med_salary'].mean(),
        'Total Views': filtered_df['views'].sum(),
        'Total Applications': filtered_df['applies'].sum(),
        'Average Normalized Salary': filtered_df['normalized_salary'].mean()
    }
    
    return result

In [8]:
print(dynamic_data_cube(merged_df, company_name= "Amazon", industry="Software Development"))

{'Total Job Listings': 343, 'Max Salary': np.float64(307900.0), 'Min Salary': np.float64(15.5), 'Average Median Salary': nan, 'Total Views': np.float64(4887.0), 'Total Applications': np.float64(293.0), 'Average Normalized Salary': np.float64(145290.2548387097)}


In [9]:
print(dynamic_data_cube(merged_df, industry= "Banking"))

{'Total Job Listings': 1304, 'Max Salary': np.float64(330000.0), 'Min Salary': np.float64(15.0), 'Average Median Salary': np.float64(17364.422), 'Total Views': np.float64(10317.0), 'Total Applications': np.float64(688.0), 'Average Normalized Salary': np.float64(88354.88512024048)}


In [10]:
print(dynamic_data_cube(merged_df, industry= "Health care"))

{'Total Job Listings': 16553, 'Max Salary': np.float64(950000.0), 'Min Salary': np.float64(10.0), 'Average Median Salary': np.float64(26060.236622418877), 'Total Views': np.float64(114444.0), 'Total Applications': np.float64(7766.0), 'Average Normalized Salary': np.float64(360982.0397253541)}


# Industry Insights Metrics

## Overview

This code calculates industry-specific metrics by grouping job postings based on the `industry` column and then aggregating key job-related statistics for each industry. The aggregated metrics include the average maximum salary, average minimum salary, total views, and total applications.

## Functionality

1. **Group by Industry**  
   The dataset (`merged_df`) is grouped by the `industry` column, so that metrics can be calculated for each industry.

2. **Aggregate Metrics**  
   For each industry, the following metrics are calculated:
   - **Average Maximum Salary** (`max_salary`): The mean of all maximum salaries for job postings in the industry.
   - **Average Minimum Salary** (`min_salary`): The mean of all minimum salaries for job postings in the industry.
   - **Total Views** (`views`): The sum of all views for job postings in the industry.
   - **Total Applications** (`applies`): The sum of all applications for job postings in the industry.

3. **Store and Display Results**  
   The results are stored in a dictionary under the key `Industry Insights`, where each industry and its associated metrics are stored as a list of dictionaries. The `result` dictionary is then printed, showing the industry-specific insights.

In [28]:
industry_metrics = merged_df.groupby('industry').agg({
    'max_salary': 'mean',
    'min_salary': 'mean',
    'views': 'sum',
    'applies': 'sum'
}).reset_index()
result = {}
result['Industry Insights'] = industry_metrics.to_dict('records')
print(result['Industry Insights'])

# Visualizing Salary Data by Industry

## Overview

This code creates a bar plot using Plotly to visualize the maximum and minimum salaries across different industries. The plot displays the industry names on the x-axis and the corresponding maximum and minimum salaries on the y-axis.

## Functionality

1. **Plot Creation with Plotly Express**  
   The `px.bar()` function is used to create a bar plot. The plot displays:
   - **x-axis**: The `industry` column, which represents different industries.
   - **y-axis**: Two columns, `max_salary` and `min_salary`, representing the maximum and minimum salary for each industry.

2. **Labels and Title**  
   - The title of the plot is set to `"Max and Min Salary by Industry"`.
   - The y-axis labels are customized to display "Max Salary" for `max_salary` and "Min Salary" for `min_salary`.

3. **Rendering the Plot**  
   The `plot(fig)` function is used to display the plot in a Jupyter Notebook or other compatible environments.

In [27]:
import plotly.express as px
import plotly.io as pio
from plotly.offline import plot
# Create a bar plot with Plotly
fig = px.bar(industry_metrics, x='industry', y=['max_salary', 'min_salary'], 
             title="Max and Min Salary by Industry", labels={'max_salary': 'Max Salary', 'min_salary': 'Min Salary'})

# To ensure Plotly works in Jupyter, you might need to explicitly render it
plot(fig)
pio.write_html(fig, file='salary_data_by_industry.html', auto_open=False)